# Crush Rig Predictive Models
* __Classifier for Trauma Score__
* __Regressor for serosa thickness delta__

Written by Matt MacDonald for CIGITI at the Hospital for Sick Children Toronto
***

All tools to manipulate data will be obtained from the crush_plot.py file. The objective of this notebook is to predict the histological targets from the force/position crush data using a classifier, either logistic regression or otherwise.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.dpi'] = 150

In [ ]:
from pdb import set_trace
from warnings import warn

The crush data must be collected using the crush rig and crush.py and stored in the expected folder structure at the root directory indicated by PATH.

In [ ]:
from crush_read import *
from crush_plot import *
PATH

Load all data and modify as needed.

In [ ]:
study = study_outline(PATH)
targets = study_targets(PATH)
crushes = study_data(study)

In [ ]:
crushes = modify(crushes)
crushes = calculate(crushes)

In [ ]:
c = random(crushes)
time_plot(c, trim=False)
time_plot(c)

Prepare data for model training and confirm no NaN issues.

In [ ]:
X, y, legend = preprocess(crushes, targets)
X.shape

In [ ]:
print('Reference for categorical features:')
legend

In [ ]:
X.isna().sum()

Remove the holding strain feature since only the STOP protocol is being considered.

In [ ]:
X = X.drop('Holding Strain', axis=1)
X.columns

Crush duration is heavily correlated with thickness because of how thickness is calculated by the position above the crush platform at the contact time.

In [ ]:
X[['Thickness (mm)', 'Crush Duration (s)']].corr()

Generate matrix of correlations to aid understanding.

In [ ]:
key_targets = y[['Percent Serosa Change', 'P Score', 'Trauma Score']].copy()
key_targets.head()

In [ ]:
pd.concat([X, key_targets], axis=1).corr()

In [ ]:
import seaborn as sns

sns.heatmap(pd.concat([X, key_targets], axis=1).corr(), center=0, vmin=-1, vmax=1, cmap='RdBu');

Interesting correlations:
- target stress is strongly correlated with trauma score and p score as expected
- target stress is not correlated with the absolute thickness change, likely because of serosal variability in patients
- strain and stiffness metrics are correlated with the thickness and tissue type of the sample
- trauma score and p score correlate, proving a relationship between the pathologist opinion and serosal thickness

Let's look at the trauma score and serosa thickness metric correlations more closely.

In [ ]:
key_targets.corr()

In [ ]:
sns.heatmap(key_targets.corr(), center=0, vmin=-1, vmax=1, cmap='RdBu');

In [ ]:
key_targets.sort_values('P Score').plot(x='P Score', y='Trauma Score', kind='scatter', alpha=0.5);
plt.yticks([0, 1, 2]);

In [ ]:
key_targets.sort_values('Percent Serosa Change').plot(x='Percent Serosa Change', y='Trauma Score', kind='scatter', alpha=0.5);
plt.yticks([0, 1, 2]);

Look for anomalies in the data.

In [ ]:
X['Crush Duration (s)'].plot(style='.');

In [ ]:
long = X['Crush Duration (s)'] > 35
long.sum()

In [ ]:
time_plot(crushes[long], trim=False)

In [ ]:
X[long]

Contact stiffness is low for this sample due to the incorrect trigger for intial contact time. The calculated thickness is also erroneously high. Most other figures are okay however so it is not a huge issue. The source of the error was the crush rig detecting contact too early as a false positive. This would have to be fixed in teh source data if desired.

In [ ]:
X[X['Thickness (mm)'] > 10]

Prepare binary classification targets.

In [ ]:
y = binary_classes(y)

In [ ]:
classifier_targets = y.columns[4:]
for col in classifier_targets:
    most_common = y[col].value_counts().idxmax()
    s = (y[col] == most_common).sum()
    c = y[col].count()
    r = s / c
    print(f"{col}\n    - baseline accuracy = {s}/{c} ({r:.2%})")

In [ ]:
X.describe()

In [ ]:
y.describe()

The major tissue damage target is unbalanced. It may not be enough data for an accurate classifier due to the skewed distribution of positive samples.

In [ ]:
plt.figure()
trauma_score = y['Tissue Damage'].copy()
trauma_score[y['Major Tissue Damage']] = 2
serosa_delta = y['Percent Serosa Change']
plt.scatter(serosa_delta, trauma_score, color='indigo')
plt.ylabel('Tissue Trauma Score')
plt.xlabel('Percent Serosa Change')
plt.yticks([0, 1, 2]);

In [ ]:
plt.figure(figsize=(5, 1))
plt.scatter(100 * y['Percent Serosa Change'], y['Tissue Damage'], color='indigo', alpha=0.25, s=100)
plt.xlabel('Serosa Thickness Change (%)')
plt.yticks([0, 1], ['No Trauma', 'Trauma'])
plt.ylim([-0.5, 1.5]);

Let's see colon vs small bowel.

In [ ]:
col_mask = X['Tissue'] == False
sb_mask = X['Tissue'] == True

In [ ]:
plt.figure(figsize=(5, 1))
plt.scatter(100 * y[col_mask]['Percent Serosa Change'], y[col_mask]['Tissue Damage'], color='indigo', alpha=0.25, s=100)
plt.xlabel('Serosa Thickness Change (%)')
plt.yticks([0, 1], ['No Trauma', 'Trauma'])
plt.ylim([-0.5, 1.5])
plt.title('Colon');

In [ ]:
plt.figure(figsize=(5, 1))
plt.scatter(100 * y[sb_mask]['Percent Serosa Change'], y[sb_mask]['Tissue Damage'], color='indigo', alpha=0.25, s=100)
plt.xlabel('Serosa Thickness Change (%)')
plt.yticks([0, 1], ['No Trauma', 'Tissue Trauma'])
plt.ylim([-0.5, 1.5])
plt.title('Small Bowel');

In [ ]:
plt.figure()
s = 0.25
m = y.shape[0]
y1 = y['Tissue Damage']
y2 = y['Significant Serosa Change']
rx = np.random.rand(m) * s - (s / 2)
ry = np.random.rand(m) * s - (s / 2)
plt.scatter(x=y1 + rx, y=y2 + ry, color='seagreen', alpha=0.25, s=100)
plt.xticks([0, 1], ['No Trauma', 'Trauma'])
plt.yticks([0, 1], ['Not Significant', 'Significant'])
plt.xlim([-0.5, 1.5])
plt.ylim([-0.5, 1.5])
plt.title('Serosa Thickness Change vs. Trauma Score')

cnts = [sum([x != y for x, y in zip(y1, y2) if x == 0]),
        sum([x == y for x, y in zip(y1, y2) if x == 1]),
        sum([x == y for x, y in zip(y1, y2) if x == 0]),
        sum([x != y for x, y in zip(y1, y2) if x == 1])]

plt.text(-0.08, 1.25, f"n={cnts[0]} ({100 * cnts[0] / m:.0f}%)", size=10)
plt.text(0.92, 1.25, f"n={cnts[1]} ({100 * cnts[1] / m:.0f}%)", size=10)
plt.text(-0.08, 0.25, f"n={cnts[2]} ({100 * cnts[2] / m:.0f}%)", size=10)
plt.text(0.92, 0.25, f"n={cnts[3]} ({100 * cnts[3] / m:.0f}%)", size=10)

print('Top left N = {} / {}'.format(cnts[0], m))
print('Top right N = {} / {}'.format(cnts[1], m))
print('Bottom left N = {} / {}'.format(cnts[2], m))
print('Bottom right N = {} / {}'.format(cnts[3], m))
print('Agreement = {} / {}'.format(sum(cnts[1:3]), m))

Visualize the key variable which is target stress. Below is the corresponding load in grams for reference.

In [ ]:
def gram_to_megapascal(load):
    return (9.81 * load) / (1000 * np.pi * (5/2)** 2)  # 5mm pin

def megapascal_to_gram(load):
    return (1000 * np.pi * (5/2)** 2) * load / 9.81  # 5mm pin

for load in [200, 400, 600, 800, 1000, 1200]:  # test loads in grams
    print(f"{gram_to_megapascal(load):6.2f} (MPa) = {load:5} (grams)")

In [ ]:
x_name = 'Target Stress (MPa)'
for y_name in y.columns:
    plt.figure()
    plt.scatter(x=X[x_name], y=y[y_name], alpha=0.25, s=100)
    plt.xlabel(x_name)
    plt.ylabel(y_name)

The goal for the prediction algorithm is to provide a metric for preventing tissue damage intraoperatively. Thus it has the following requirements:

1. Good overall accuracy so it is reliable without being restrictive
2. High recall such that it is conservative, limiting the occurrence of false negatives
3. Simple with limited input so that it can be implemented cheaply in real time

Further to requirement 3 above, no histology features can be used to make the prediction.

In [ ]:
def get_freq(crush):
    time = crush.index
    delta = time[1:] - time[:-1]
    return 1 / np.mean(delta.total_seconds())

freqs = crushes['Data'].apply(get_freq)
freqs.mean()

Sample frequency is 31 Hz

Nyquist frequency is 62 Hz

Cutoff frequency of 3rd order butterworth digital filter is 0.2 * 62 = 12.4 Hz

# Data Prep
Define the targets and set the random seed for the model training.

In [ ]:
SEED = 0

In [ ]:
y.columns

In [ ]:
targets = ['Percent Serosa Change',  # regression
           'Significant Serosa Change',
           'Tissue Damage',
           'Major Tissue Damage']

class_labels = {'Percent Serosa Change': None,
                'Significant Serosa Change': ['No Change', 'Significant Change'],
                'Tissue Damage': ['No Damage', 'Damage'],
                'Major Tissue Damage': ['No Damage or Minor Damage', 'Major Damage']}

In [ ]:
for i, targ in enumerate(targets):
    print(i, targ)

Split the data.

In [ ]:
from sklearn.model_selection import train_test_split

X_np = X.values.astype(np.float64)
y_np = y[targets].values

X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=SEED)

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Standardizing the features makes the final model harder to apply and interpret manually. So it is not advisable in this case.

In [ ]:
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
pd.DataFrame(X_train).boxplot();

In [ ]:
features = X.columns.values
for i, feat in enumerate(features):
    print(i, feat)
stress_feature = features[6]

# Model Prep
Create functions needed for fitting predictive models to the data.

For classification we will use logistic regression due to it's long standing success as a binary classificaiton model for medical data. For regression we will rely on lasso regression to take advantage of the feature selection behaviour given the number of low correlation features in the dataset. Both are also simple enough to be practical for hand calculations or lookup tables in the OR.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

def logit_reg(X, y):
    # Fit classifier using cross validation and l1 loss
    model = LogisticRegressionCV(penalty='l1',
                                 Cs=10,
                                 cv=5,
                                 scoring='accuracy',
                                 solver='liblinear',
                                 random_state=SEED,
                                 max_iter=2500,
                                 n_jobs=4)
    return model.fit(X, y)

In [ ]:
from sklearn.linear_model import LassoCV

def lasso_reg(X, y):
    # Fit regressor using cross validation and l1 loss
    model = LassoCV(n_alphas=10,
                    cv=5,
                    random_state=SEED,
                    max_iter=2500,
                    n_jobs=4)
    return model.fit(X, y)

In [ ]:
from sklearn.feature_selection import RFE, RFECV

def optimal_reg(X, y, target, n_features=None):
    # Restrict the features used to get a simpler model
    
    # Lasso and MSE for regression
    if class_labels[target] is None:
        model = lasso_reg(X, y)
        if n_features is not None:
            rfe = RFE(model, n_features_to_select=n_features)
        else:
            rfe = RFECV(model, cv=5, scoring='neg_mean_squared_error', n_jobs=4)
    
    # Logistic regression and accuracy for classification
    else:
        model = logit_reg(X, y)
        if n_features is not None:
            rfe = RFE(model, n_features_to_select=n_features)
        else:
            rfe = RFECV(model, cv=5, scoring='accuracy', n_jobs=4)
    
    return rfe.fit(X, y)

Create metric review functions.

In [ ]:
from sklearn.metrics import classification_report, mean_squared_error

def metrics(X, y, model, target):
    # RMSE and fit curve for regressors
    if class_labels[target] is None:
        plot_fit(X, y, model, target)
        return {'RMSE': np.sqrt(mean_squared_error(y, model.predict(X)))}
        
    
    # Classification metrics and AUC for classifiers
    plot_auc(X, y, model, target)
    report = classification_report(y, model.predict(X),
                                   digits=3,
                                   output_dict=True)
#     for i, num in enumerate(np.unique(y)):
#         report[class_labels[target][i]] = report.pop(str(num))
    return report['1.0']  # positive calss only

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

def plot_auc(X, y, model, target):
    plt.figure()
    plt.plot(*roc_curve(y, model.predict_proba(X)[:, 1])[:2],
             label=f"ROC curve (AUC = {roc_auc_score(y, model.predict(X)):.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.title(target)
    plt.show()

In [ ]:
def plot_fit(X, y, model, target):
    best_feat = np.argmax(np.abs(model.coef_)) 
    plt.figure()
    plt.plot(X[:, best_feat], model.predict(X), 'k.')
    plt.scatter(X[:, best_feat], y, alpha=0.25, s=100)
    plt.xlabel(stress_feature)  # assumed to be best feature
    plt.ylabel(target)
    plt.legend(['Predicted', 'Actual'])
    plt.show()

In [ ]:
def build(X, y, model, n_features=None):
    '''
    Convenient function  to build multiple models
    Remove any features deemed to be irrelevent by recursive feature elimination
    '''
    if n_features is None:
        n_features = X.shape[1]
    rfe = restricted_reg(X, y, model, n_features)
    
    # Rank the features
    rank = pd.DataFrame({'features': features,
                         'ranking': rfe.ranking_})
    rank = rank.sort_values(by='ranking')
    selected = rank.features[:n_features].tolist()
    
    # Train the model once more on just the selected features
    mask = rfe.support_
    model = logit_reg(X[:, mask], y)
    
    return {'model': model,
            'rank': rank,
            'features': selected,
            'n_features': n_features}

# Regression (Linear or Logistic)

Build linear regression model for the continuous targets. Build logistic regression models for the class target values.

### Percent Change in Serosa Thickness
Regression model with Lasso.

In [ ]:
ind = 0
targets[ind]

In [ ]:
model = lasso_reg(X_train, y_train[:, ind])

In [ ]:
metrics(X_train, y_train[:, ind], model, targets[ind])

In [ ]:
metrics(X_test, y_test[:, ind], model, targets[ind])

In [ ]:
rfe = optimal_reg(X_train, y_train[:, ind], targets[ind])

In [ ]:
print(features[rfe.support_])

In [ ]:
metrics(X_train[:, rfe.support_], y_train[:, ind], rfe.estimator_, targets[ind])

In [ ]:
metrics(X_test[:, rfe.support_], y_test[:, ind], rfe.estimator_, targets[ind])

In [ ]:
rank = pd.DataFrame({'features': features,
                     'ranking': rfe.ranking_})
rank = rank.sort_values(by='ranking')
rank

### Significant Change in Serosa Thickness

In [ ]:
ind = 1
targets[ind]

In [ ]:
model = logit_reg(X_train, y_train[:, ind])

In [ ]:
metrics(X_train, y_train[:, ind], model, targets[ind])

In [ ]:
metrics(X_test, y_test[:, ind], model, targets[ind])

In [ ]:
rfe = optimal_reg(X_train, y_train[:, ind], targets[ind])

In [ ]:
print(features[rfe.support_])

In [ ]:
metrics(X_train[:, rfe.support_], y_train[:, ind], rfe.estimator_, targets[ind])

In [ ]:
metrics(X_test[:, rfe.support_], y_test[:, ind], rfe.estimator_, targets[ind])

In [ ]:
rank = pd.DataFrame({'features': features,
                     'ranking': rfe.ranking_})
rank = rank.sort_values(by='ranking')
rank

Let's check how much the performance deteriorates with only one feature.

In [ ]:
rfe = optimal_reg(X_train, y_train[:, ind], targets[ind], n_features=1)

In [ ]:
metrics(X_train[:, rfe.support_], y_train[:, ind], rfe.estimator_, targets[ind])

In [ ]:
metrics(X_test[:, rfe.support_], y_test[:, ind], rfe.estimator_, targets[ind])

**No measureable difference!**

Review the model parameters.

In [ ]:
model = rfe.estimator_
print('Model coefficients:')
print(model.coef_)
print('Model features:')
print(features[rfe.support_])
print('Model intercept:')
print(model.intercept_)
print('Model regularization:')
C = model.C_
print(f"C = {C}, lambda = {1 / C}")

Where does the model predict significant serosa thickness change then?

In [ ]:
stress = np.linspace(0, 1, 1000).reshape(-1, 1)
pred = model.predict(stress)
prob = model.predict_proba(stress)

In [ ]:
print(megapascal_to_gram(stress[np.argmax(pred)]), 'gram limit')

In [ ]:
plt.scatter(stress, pred)
plt.scatter(stress, prob[:, 1]);

### Tissue Damage

In [ ]:
ind = 2
targets[ind]

In [ ]:
model = logit_reg(X_train, y_train[:, ind])

In [ ]:
metrics(X_train, y_train[:, ind], model, targets[ind])

In [ ]:
metrics(X_test, y_test[:, ind], model, targets[ind])

In [ ]:
rfe = optimal_reg(X_train, y_train[:, ind], targets[ind])

In [ ]:
print(features[rfe.support_])

In [ ]:
metrics(X_train[:, rfe.support_], y_train[:, ind], rfe.estimator_, targets[ind])

In [ ]:
metrics(X_test[:, rfe.support_], y_test[:, ind], rfe.estimator_, targets[ind])

In [ ]:
rank = pd.DataFrame({'features': features,
                     'ranking': rfe.ranking_})
rank = rank.sort_values(by='ranking')
rank

Let's check how much the performance deteriorates with only one feature.

In [ ]:
rfe = optimal_reg(X_train, y_train[:, ind], targets[ind], n_features=1)

In [ ]:
metrics(X_train[:, rfe.support_], y_train[:, ind], rfe.estimator_, targets[ind])

In [ ]:
metrics(X_test[:, rfe.support_], y_test[:, ind], rfe.estimator_, targets[ind])

**Not a huge loss for the training set and the test set actually performs better..**

Review the model parameters.

In [ ]:
model = rfe.estimator_
print('Model coefficients:')
print(model.coef_)
print('Model features:')
print(features[rfe.support_])
print('Model intercept:')
print(model.intercept_)
print('Model regularization:')
C = model.C_
print(f"C = {C}, lambda = {1 / C}")

Where does the model predict tissue damage then?

In [ ]:
stress = np.linspace(0, 1, 1000).reshape(-1, 1)
pred = model.predict(stress)
prob = model.predict_proba(stress)

In [ ]:
print(megapascal_to_gram(stress[np.argmax(pred)]), 'gram limit')

In [ ]:
plt.scatter(stress, pred)
plt.scatter(stress, prob[:, 1]);

### Major Tissue Damage

Select a specific indicator from the targets and split the dataset.

In [ ]:
ind = 3
targets[ind]

In [ ]:
model = logit_reg(X_train, y_train[:, ind])

In [ ]:
y_train[:, ind].sum()

Only 4 positive examples for major damage!! That is far too few to build a useable model as feared.

In [ ]:
features[6]

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', max_iter=2500, n_jobs=4, random_state=SEED)
model.fit(X_train[:, 6].reshape(-1, 1), y_train[:, ind])

In [ ]:
mostress = np.linspace(0, 1, 1000).reshape(-1, 1)
pred = model.predict(stress)
prob = model.predict_proba(stress)

In [ ]:
print(megapascal_to_gram(stress[np.argmax(pred)]), 'gram limit')

In [ ]:
plt.scatter(stress, pred)
plt.scatter(stress, prob[:, 1]);